In [10]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from keras.layers import Bidirectional
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import datetime
from numpy import hstack, vstack
from numpy import array
from sklearn.linear_model import SGDClassifier as SGD
from keras.models import load_model

Uses multi vairable LSTM model

In [11]:
model = load_model('Multiple_Parallel.h5')

Metal device set to: Apple M2


2022-11-13 19:46:47.936093: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-13 19:46:47.938108: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [18]:
#obtain list of dates
directory = "Boston Celtics"
list_dates = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        list_dates.append(filename)

In [19]:
# create dictionary 
final_data = {}

In [20]:
# for minutes

minutes = {}

opponents = {}
# reads in each csv
for i in list_dates:
    data = pd.read_csv(f"{directory}/{i}", index_col=0)
    #gathers data and adds to dict
    date = data["Date"].tolist()[0]

    min_data = data.copy()



    minute = data["MIN"].tolist()

    minute_array = []

    for i in minute:
        time = i.split(":")

        minute_array.append(int(time[0]) + (int(time[0]) / 60))

    # (error for now) opponents[date] = data["Opponent"][0]

    data["MIN"] = minute_array

    data[date] = data["Score"]
    min_data[date] = data["MIN"]

    final_data[date] = data[date]
    minutes[date] = min_data[date]

# convert dict to dataframe
data_df = pd.DataFrame(final_data)

# convert dict to dataframe
min_df = pd.DataFrame(minutes)

opp = pd.DataFrame.transpose(pd.DataFrame.from_dict(opponents, orient="index"))
data_df = data_df.append(opp)

tester = pd.DataFrame.transpose(data_df)

tester["Datetime"] = pd.to_datetime(data_df.columns.tolist()).tolist()

tester = tester.sort_values(by="Datetime")

columns = tester.columns.tolist()[:-1]

tester_min = pd.DataFrame.transpose(min_df)

tester_min["Datetime"] = pd.to_datetime(min_df.columns.tolist()).tolist()

tester_min = tester_min.sort_values(by="Datetime")

columns = tester_min.columns.tolist()[:-1]

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_42789/1099516675.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(opp)


In [21]:
add = {}

values = {}

diff = {}

columns = tester.columns.tolist()[:-1]

#iterate through index
for column in columns:
    
    if column not in add:
        add[column] = []
        
        values[column] = []
        
        diff[column] = []
    
    print(column)
    
    # split sequence
    def split_sequence(sequences, n_steps):
        X, y = list(), list()
        for i in range(len(sequences)):
            # find end of pattern 
            end_ix = i + n_steps
            #check if beyond sequence to break
            if end_ix > len(sequences) - 1:
                break
            #gather input and output parts of the pattern
            seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)
        # sequence
    
        # sequence
    df = tester[[column]]
    
    df["Min"] = tester_min[column].tolist()
    
    name = df.columns[0]
    
    rent_rate = array(df[name])
    
    # number of steps
    rent_rate = array(tester[column].dropna())
    predict = rent_rate
    eff = array(df["Min"].dropna())
    
    
    n_steps = 3
    if len(rent_rate) < 5:
        continue   
    
    
    # convert to [rows, columns] structure
    rent_rate = rent_rate.reshape((len(rent_rate), 1))
    eff = eff.reshape((len(eff), 1))
    predict = predict.reshape((len(predict), 1))
    
    # horizontally stack columns
    dataset = hstack((rent_rate, eff))    
        
    X, y = split_sequence(dataset, n_steps)
    
    X = X.astype(float)
    # summarize the data
    for i in range(len(X)):
        print(X[i], y[i])
        
    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = X.shape[2]
    
    rent = rent_rate[-3:]
    
    eff_pred = eff[-3:]
    
    # demonstrate prediction
    x_input = array([[rent[0], eff_pred[0]], [rent[1], eff_pred[1]], [rent[2], eff_pred[2]]])
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    add[column].append(yhat.tolist()[0][0])

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_42789/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Min"] = tester_min[column].tolist()


Al Horford
[[14.75       23.38333333]
 [16.75       33.55      ]
 [20.25       25.41666667]] [23.         41.68333333]
[[16.75       33.55      ]
 [20.25       25.41666667]
 [23.         41.68333333]] [18.5        23.38333333]
[[20.25       25.41666667]
 [23.         41.68333333]
 [18.5        23.38333333]] [32.5  39.65]
[[23.         41.68333333]
 [18.5        23.38333333]
 [32.5        39.65      ]] [26.75 33.55]
[[18.5        23.38333333]
 [32.5        39.65      ]
 [26.75       33.55      ]] [33.         31.51666667]
[[32.5        39.65      ]
 [26.75       33.55      ]
 [33.         31.51666667]] [26.         26.43333333]
[[26.75       33.55      ]
 [33.         31.51666667]
 [26.         26.43333333]] [37.75       34.56666667]
Blake Griffin
[[8.75       8.13333333]
 [3.5        9.15      ]
 [8.5        5.08333333]] [11.         11.18333333]
[[ 3.5         9.15      ]
 [ 8.5         5.08333333]
 [11.         11.18333333]] [10.75 15.25]
[[ 8.5         5.08333333]
 [11.         11.1

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_42789/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Min"] = tester_min[column].tolist()
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_42789/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Min"] = tester_min[column].tolist()
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_42789/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

Grant Williams
[[19.25       23.38333333]
 [26.25       25.41666667]
 [29.         31.51666667]] [ 6.75 18.3 ]
[[26.25       25.41666667]
 [29.         31.51666667]
 [ 6.75       18.3       ]] [21.25 24.4 ]
[[29.         31.51666667]
 [ 6.75       18.3       ]
 [21.25       24.4       ]] [26.75       31.51666667]
[[ 6.75       18.3       ]
 [21.25       24.4       ]
 [26.75       31.51666667]] [12.5        20.33333333]
[[21.25       24.4       ]
 [26.75       31.51666667]
 [12.5        20.33333333]] [22.5  27.45]
[[26.75       31.51666667]
 [12.5        20.33333333]
 [22.5        27.45      ]] [12.75       37.61666667]
[[12.5        20.33333333]
 [22.5        27.45      ]
 [12.75       37.61666667]] [12.         20.33333333]
[[22.5        27.45      ]
 [12.75       37.61666667]
 [12.         20.33333333]] [29.         32.53333333]
[[12.75       37.61666667]
 [12.         20.33333333]
 [29.         32.53333333]] [38.5        44.73333333]
JD Davison
Jaylen Brown
[[47.75       38.63333333

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_42789/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Min"] = tester_min[column].tolist()
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_42789/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Min"] = tester_min[column].tolist()
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_42789/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

Payton Pritchard
[[3.5        3.05      ]
 [7.75       5.08333333]
 [3.         7.11666667]] [ 8.25 12.2 ]
[[ 7.75        5.08333333]
 [ 3.          7.11666667]
 [ 8.25       12.2       ]] [18.25 15.25]
[[ 3.          7.11666667]
 [ 8.25       12.2       ]
 [18.25       15.25      ]] [10.         14.23333333]
Sam Hauser
[[ 0.    3.05]
 [ 9.   12.2 ]
 [13.    6.1 ]] [ 8.5        11.18333333]
[[ 9.         12.2       ]
 [13.          6.1       ]
 [ 8.5        11.18333333]] [ 6.25       11.18333333]
[[13.          6.1       ]
 [ 8.5        11.18333333]
 [ 6.25       11.18333333]] [23.75       22.36666667]
[[ 8.5        11.18333333]
 [ 6.25       11.18333333]
 [23.75       22.36666667]] [ 0.         10.16666667]
[[ 6.25       11.18333333]
 [23.75       22.36666667]
 [ 0.         10.16666667]] [14.  18.3]
[[23.75       22.36666667]
 [ 0.         10.16666667]
 [14.         18.3       ]] [26.25 21.35]
[[ 0.         10.16666667]
 [14.         18.3       ]
 [26.25       21.35      ]] [10.25    

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_42789/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Min"] = tester_min[column].tolist()
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_42789/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Min"] = tester_min[column].tolist()


In [22]:
# convert dict to dataframe
Data = pd.DataFrame.from_dict(add, orient='index')
Data

,0
Al Horford,22.815506
Blake Griffin,10.521413
Derrick White,8.215691
Grant Williams,28.166260
JD Davison,NaN
Jaylen Brown,48.144436
Jayson Tatum,26.725595
Justin Jackson,NaN
Luke Kornet,17.557364
Malcolm Brogdon,20.998320


In [23]:
# convert dataframe to csv
mypath = "Final_Score/LSTM"
if not os.path.isdir(mypath):
    os.makedirs(mypath)
Data.to_csv(f"{mypath}/{directory}.csv", index=True)